In [1]:
from IPython.display import display, HTML 
display(HTML("""
<style>
div.container{width:86% !important;}
div.cell.code_cell.rendered{width:100%;}
div.CodeMirror {font-family:Consolas; font-size:15pt;}
div.output {font-size:15pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:15pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:15px;}
</style>
"""))

<b><font color='red' size='6'>ch12. 데이터프레임과 시리즈(pandas)</font></b>

# 9절 데이터 그룹화 및 집계

In [8]:
import seaborn as sns
iris = sns.load_dataset('iris')
iris.iloc[::25]

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
25,5.0,3.0,1.6,0.2,setosa
50,7.0,3.2,4.7,1.4,versicolor
75,6.6,3.0,4.4,1.4,versicolor
100,6.3,3.3,6.0,2.5,virginica
125,7.2,3.2,6.0,1.8,virginica


## 9.1 groupby
cf. pivot_table

### 1) 그룹화 방법1
- df.gropby(by=[그룹핑할열]).기초통계함수(sum,mean,std...)
-    [함수적용해서 출력할 열들] 생략시 그룹핑할열을 제외한 모든 열 적용

In [56]:
iris_grouped = iris.groupby('species')
iris_grouped = iris.groupby(iris.species)
#iris_grouped = iris.groupby(['species',])
for group, data in iris_grouped:
    print(group)
    display(data.head(2).reset_index())

setosa


,index,sepal_length,sepal_width,petal_length,petal_width,species
0,0,5.1,3.5,1.4,0.2,setosa
1,1,4.9,3.0,1.4,0.2,setosa


versicolor


,index,sepal_length,sepal_width,petal_length,petal_width,species
0,50,7.0,3.2,4.7,1.4,versicolor
1,51,6.4,3.2,4.5,1.5,versicolor


virginica


,index,sepal_length,sepal_width,petal_length,petal_width,species
0,100,6.3,3.3,6.0,2.5,virginica
1,101,5.8,2.7,5.1,1.9,virginica


In [15]:
iris.groupby(by='species').mean()[['petal_length','petal_width']]
iris.groupby(by='species')[['petal_length','petal_width']].mean()

,petal_length,petal_width
species,,
setosa,1.462,0.246
versicolor,4.260,1.326
virginica,5.552,2.026


In [50]:
iris_grouped_sum = iris.groupby('species').median()
iris_grouped_sum.reset_index()

,species,sepal_length,sepal_width,petal_length,petal_width
0,setosa,5.0,3.4,1.50,0.2
1,versicolor,5.9,2.8,4.35,1.3
2,virginica,6.5,3.0,5.55,2.0


In [55]:
iris_grouped_sum2 = iris.groupby('species', as_index=False).sum()
iris_grouped_sum2

,species,sepal_length,sepal_width,petal_length,petal_width
0,setosa,250.3,171.4,73.1,12.3
1,versicolor,296.8,138.5,213.0,66.3
2,virginica,329.4,148.7,277.6,101.3


In [54]:
iris_grouped_sum2[(iris_grouped_sum2.species == 'setosa')|(iris_grouped_sum2.species == 'versicolor')]

,species,sepal_length,sepal_width,petal_length,petal_width
0,setosa,250.3,171.4,73.1,12.3
1,versicolor,296.8,138.5,213.0,66.3


### 2) 그룹화 방법 2
- df.pivot_table( index=[그룹핑할열], values=[함수적용해서 출력할 열들], aggfunc=함수(defualt:mean) )

In [59]:
# iris.groupby('species')['petal_length','petal_width'].mean()

iris.pivot_table(index='species', 
                 values=['petal_length','petal_width'],
                 aggfunc='mean')

,petal_length,petal_width
species,,
setosa,1.462,0.246
versicolor,4.260,1.326
virginica,5.552,2.026


In [60]:
iris.groupby('species').count()

,sepal_length,sepal_width,petal_length,petal_width
species,,,,
setosa,50,50,50,50
versicolor,50,50,50,50
virginica,50,50,50,50


In [62]:
iris.pivot_table(index='species', aggfunc='count')

,petal_length,petal_width,sepal_length,sepal_width
species,,,,
setosa,50,50,50,50
versicolor,50,50,50,50
virginica,50,50,50,50


### 3) 다중열로 그룹화

In [63]:
iris.shape

(150, 5)

In [73]:
import numpy as np
np.ravel([[i]*25 for i in range(6)])

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5])

In [76]:
iris['num'] = np.ravel([[i]*25 for i in range(6)])
iris[::10]

,sepal_length,sepal_width,petal_length,petal_width,species,num
0,5.1,3.5,1.4,0.2,setosa,0
10,5.4,3.7,1.5,0.2,setosa,0
20,5.4,3.4,1.7,0.2,setosa,0
30,4.8,3.1,1.6,0.2,setosa,1
40,5.0,3.5,1.3,0.3,setosa,1
50,7.0,3.2,4.7,1.4,versicolor,2
60,5.0,2.0,3.5,1.0,versicolor,2
70,5.9,3.2,4.8,1.8,versicolor,2
80,5.5,2.4,3.8,1.1,versicolor,3
90,5.5,2.6,4.4,1.2,versicolor,3


In [79]:
# 각열의 임의의 데이터에 결측치를 할당
iris.head(2)

,sepal_length,sepal_width,petal_length,petal_width,species,num
0,5.1,3.5,1.4,0.2,setosa,0
1,4.9,3.0,1.4,0.2,setosa,0


In [106]:
import random
print(random.sample(range(149),3))
for col in range(iris.shape[1]-2):
    iris.iloc[random.sample(range(150),col+1),col] = np.nan

[37, 95, 31]


In [115]:
iris.groupby(by=['species','num'])[['petal_length','petal_width']].count()

petal_length  petal_width
species    num                           
setosa     0              25           25
           1              25           24
versicolor 2              24           25
           3              25           23
virginica  4              25           25
           5              23           24

In [117]:
iris.pivot_table(index=['species','num'], values=['petal_length','petal_width'], aggfunc='count')

petal_length  petal_width
species    num                           
setosa     0              25           25
           1              25           24
versicolor 2              24           25
           3              25           23
virginica  4              25           25
           5              23           24

In [146]:
iris.groupby('species').describe().T

species                setosa  versicolor  virginica
sepal_length count  50.000000   49.000000  50.000000
             mean    5.006000    5.946939   6.588000
             std     0.352490    0.515632   0.635880
             min     4.300000    4.900000   4.900000
             25%     4.800000    5.600000   6.225000
             50%     5.000000    5.900000   6.500000
             75%     5.200000    6.300000   6.900000
             max     5.800000    7.000000   7.900000
sepal_width  count  49.000000   50.000000  49.000000
             mean    3.432653    2.770000   2.973469
             std     0.381547    0.313798   0.325817
             min     2.300000    2.000000   2.200000
             25%     3.200000    2.525000   2.800000
             50%     3.400000    2.800000   3.000000
             75%     3.700000    3.000000   3.200000
             max     4.400000    3.400000   3.800000
petal_length count  50.000000   49.000000  48.000000
             mean    1.462000    4.255102   5.558333
             std     0.173664    0.473489   0.553070
             min     1.000000    3.000000   4.500000
             25%     1.400000    4.000000   5.100000
             50%     1.500000    4.300000   5.550000
             75%     1.575000    4.600000   5.825000
             max     1.900000    5.100000   6.900000
petal_width  count  49.000000   48.000000  49.000000
             mean    0.246939    1.318750   2.020408
             std     0.106266    0.194232   0.274605
             min     0.100000    1.000000   1.400000
             25%     0.200000    1.200000   1.800000
             50%     0.200000    1.300000   2.000000
             75%     0.300000    1.500000   2.300000
             max     0.600000    1.800000   2.500000
num          count  50.000000   50.000000  50.000000
             mean    0.500000    2.500000   4.500000
             std     0.505076    0.505076   0.505076
             min     0.000000    2.000000   4.000000
             25%     0.000000    2.000000   4.000000
             50%     0.500000    2.500000   4.500000
             75%     1.000000    3.000000   5.000000
             max     1.000000    3.000000   5.000000

## 9.2 레이블 인코딩

In [148]:
iris.drop('num',axis=1,inplace=True)
iris

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


In [159]:
# 레이블 인코딩을 지원하는 클래스 import 
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder() # 객체
iris['target'] = le.fit_transform(iris.species)
iris[::25]

,sepal_length,sepal_width,petal_length,petal_width,species,target
0,5.1,3.5,1.4,0.2,setosa,0
25,5.0,3.0,1.6,0.2,setosa,0
50,7.0,3.2,4.7,1.4,versicolor,1
75,6.6,3.0,4.4,1.4,versicolor,1
100,6.3,3.3,6.0,2.5,virginica,2
125,7.2,3.2,6.0,1.8,virginica,2


# 10절 데이터 구조 변경
- 와이드 포맷 <-> 롱포맷
- [판다스 공식 문서의 melt](https://pandas.pydata.org/docs/user_guide/reshaping.html#reshaping-by-melt)

In [161]:
from statsmodels.api import datasets
airquality = datasets.get_rdataset('airquality', package='datasets').data
airquality.head(2)

,Ozone,Solar.R,Wind,Temp,Month,Day
0,41.0,190.0,7.4,67,5,1
1,36.0,118.0,8.0,72,5,2


## 10.1 melt를 이용한 언피벗팅(와이드포맷 -> 롱포맷)

In [218]:
airquality_melted =  airquality.melt(id_vars=['Month','Day'],
                                     var_name='변수명',
                                     value_name='값').sort_values(['Month','Day'])

In [219]:
airquality_melted

,Month,Day,변수명,값
0,5,1,Ozone,41.0
153,5,1,Solar.R,190.0
306,5,1,Wind,7.4
459,5,1,Temp,67.0
1,5,2,Ozone,36.0
...,...,...,...,...
610,9,29,Temp,76.0
152,9,30,Ozone,20.0
305,9,30,Solar.R,223.0
458,9,30,Wind,11.5


## 10.2 pivot_table을 이용한 피벗팅(롱포맷->와이드포맷)

In [216]:
# airquality_melted를 와이드포맷으로
airquality2 = airquality_melted.pivot_table(index=['Month','Day'],
                              columns='변수명', 
                              values='값')
airquality2.reset_index(inplace=True)
airquality2.columns.name = None

In [217]:
airquality2

,Month,Day,Ozone,Solar.R,Temp,Wind
0,5,1,41.0,190.0,67.0,7.4
1,5,2,36.0,118.0,72.0,8.0
2,5,3,12.0,149.0,74.0,12.6
3,5,4,18.0,313.0,62.0,11.5
4,5,5,NaN,NaN,56.0,14.3
...,...,...,...,...,...,...
148,9,26,30.0,193.0,70.0,6.9
149,9,27,NaN,145.0,77.0,13.2
150,9,28,14.0,191.0,75.0,14.3
151,9,29,18.0,131.0,76.0,8.0


# 11절 데이터프레임(시리즈)에 함수적용시키기
- apply(func, axis) : 축요서별(요소별) 함수 적용 (데이터프레임이나 시리즈에서 함수 적용)
- applymap(func) : 각 요소별 함수 적용(데이터프레임에서만 가능)
- map(func, list) : 각 요소별 함수 적용(1차원 리스트, 시리즈에서만 가능)

## 11.1 apply

In [230]:
member = pd.read_csv('data/ch11_member.csv', comment='#')
member

,Name,Age,Email,Address,Birth
0,홍길동,20,hong@hong.com,서울시 강동구,1999-01-01
1,홍길서,30,seo@hong.com,서울시 중구,1999-12-01
2,홍길남,21,nam@hong.com,서울시 강동구,1999-11-01
3,홍길북,22,buk@hong.com,서울시 관악구,1999-05-01
4,홍길중,23,mid@hong.com,부산시 중구,1999-01-30


In [238]:
get_year = lambda datestr : int(datestr.split('-')[0])
get_year('1999-05-01')

1999

In [237]:
get_month = lambda datestr : int(datestr.split('-')[1])
get_month('1999-05-01')

5

In [242]:
member['year'] = member['Birth'].apply(get_year)

In [243]:
member

,Name,Age,Email,Address,Birth,year
0,홍길동,20,hong@hong.com,서울시 강동구,1999-01-01,1999
1,홍길서,30,seo@hong.com,서울시 중구,1999-12-01,1999
2,홍길남,21,nam@hong.com,서울시 강동구,1999-11-01,1999
3,홍길북,22,buk@hong.com,서울시 관악구,1999-05-01,1999
4,홍길중,23,mid@hong.com,부산시 중구,1999-01-30,1999


In [248]:
X = iris.iloc[:,:-2]
# X 의 열별 평균

In [257]:
avg = X.mean(axis=0)
X.iloc[0] - avg

sepal_length   -0.746309
sepal_width     0.443243
petal_length   -2.330612
petal_width    -0.994521
dtype: float64

In [265]:
# X의 모든행에 대하여 열별 평균과의 거리를 소수점 2자리까지 출력
X.apply(lambda x : x**2, axis=0)

,sepal_length,sepal_width,petal_length,petal_width
0,26.01,12.25,1.96,0.04
1,24.01,9.00,1.96,0.04
2,22.09,10.24,1.69,0.04
3,21.16,9.61,2.25,0.04
4,25.00,12.96,1.96,0.04
...,...,...,...,...
145,44.89,9.00,27.04,5.29
146,39.69,6.25,25.00,3.61
147,42.25,9.00,27.04,4.00
148,38.44,11.56,29.16,5.29


## 11.2 applymap

In [262]:
X.applymap(np.round)

,sepal_length,sepal_width,petal_length,petal_width
0,5.0,4.0,1.0,0.0
1,5.0,3.0,1.0,0.0
2,5.0,3.0,1.0,0.0
3,5.0,3.0,2.0,0.0
4,5.0,4.0,1.0,0.0
...,...,...,...,...
145,7.0,3.0,5.0,2.0
146,6.0,2.0,5.0,2.0
147,6.0,3.0,5.0,2.0
148,6.0,3.0,5.0,2.0


## 11.3 map
- 1차원

In [267]:
member['month']= member['Birth'].map(lambda datestr : int(datestr.split('-')[1]))
member

,Name,Age,Email,Address,Birth,year,month
0,홍길동,20,hong@hong.com,서울시 강동구,1999-01-01,1999,1
1,홍길서,30,seo@hong.com,서울시 중구,1999-12-01,1999,12
2,홍길남,21,nam@hong.com,서울시 강동구,1999-11-01,1999,11
3,홍길북,22,buk@hong.com,서울시 관악구,1999-05-01,1999,5
4,홍길중,23,mid@hong.com,부산시 중구,1999-01-30,1999,1


In [280]:
new_df['연도'] = new_df['연도월'].apply(lambda x : int(x.split('년')[0]))
new_df['월'] = new_df['연도월'].apply(lambda x : int(x.split('년')[1][:-1]))
new_df.drop('연도월' , axis=1)

,지역,평당분양가,연도,월
0,서울,18189,2013,12
1,부산,8111,2013,12
2,대구,8080,2013,12
3,인천,10204,2013,12
4,광주,6098,2013,12
...,...,...,...,...
352,전북,6580,2015,8
353,전남,6289,2015,8
354,경북,7037,2015,8
355,경남,7665,2015,8


In [283]:
new_df.head(2)

,지역,연도월,평당분양가
0,서울,2013년12월,18189
1,부산,2013년12월,8111


In [316]:
def year_month_append(df):
    'df를 받아 연도와 월 추가후 return'
    df = df.copy()
    df['연도'] = int(df['연도월'].split('년')[0])
    df['월'] = int(df['연도월'].split('년')[1][:-1])
    return df.drop('연도월')

In [317]:
new_df.apply(year_month_append, axis=1)

,지역,평당분양가,연도,월
0,서울,18189,2013,12
1,부산,8111,2013,12
2,대구,8080,2013,12
3,인천,10204,2013,12
4,광주,6098,2013,12
...,...,...,...,...
352,전북,6580,2015,8
353,전남,6289,2015,8
354,경북,7037,2015,8
355,경남,7665,2015,8


In [305]:
new_df

,지역,연도월,평당분양가
0,서울,2013년12월,18189
1,부산,2013년12월,8111
2,대구,2013년12월,8080
3,인천,2013년12월,10204
4,광주,2013년12월,6098
...,...,...,...
352,전북,2015년8월,6580
353,전남,2015년8월,6289
354,경북,2015년8월,7037
355,경남,2015년8월,7665


# 12절 일괄변경(결측치, 특정값)
## 12.1 fillna
- 대부분 결측치는 drop하거나, 평균값대체, 중위수대체, 그룹별 평균값, 그룹별 중위수, 이전값, 이후값 등으로 대체

In [319]:
df = pd.DataFrame([[np.nan, 2, np.nan, 0],
                   [3,      4, np.nan, 1],
                   [np.nan, 3, np.nan, 5],
                   [np.nan, 3, np.nan, 4]], columns=[list('ABCD')])
df

,A,B,C,D
0,NaN,2,NaN,0
1,3.0,4,NaN,1
2,NaN,3,NaN,5
3,NaN,3,NaN,4


In [326]:
# 모든 결측치를 0 대체
df.fillna(0)

,A,B,C,D
0,0.0,2,0.0,0
1,3.0,4,0.0,1
2,0.0,3,0.0,5
3,0.0,3,0.0,4


In [328]:
df.fillna(method='ffill') # ffill 이전 행의 값으로 대체, 없다면 NaN

,A,B,C,D
0,NaN,2,NaN,0
1,3.0,4,NaN,1
2,3.0,3,NaN,5
3,3.0,3,NaN,4


In [329]:
df.fillna(method='bfill')# bfill : 다음행의 값으로 대체

,A,B,C,D
0,3.0,2,NaN,0
1,3.0,4,NaN,1
2,NaN,3,NaN,5
3,NaN,3,NaN,4


In [330]:
# 결측치를 특정값으로 대체
values = {'A':85,'B':80,'C':75,'D':90}
df.fillna(values) 

,A,B,C,D
0,85.0,2,75.0,0
1,3.0,4,75.0,1
2,85.0,3,75.0,5
3,85.0,3,75.0,4


In [332]:
# 결측치를 열별 평균값으로 대체
df.fillna(df.mean())

,A,B,C,D
0,3.0,2,NaN,0
1,3.0,4,NaN,1
2,3.0,3,NaN,5
3,3.0,3,NaN,4


In [335]:
# 결측치를 2번째 결측치까지만 대체 
df.fillna(values, limit=2)

,A,B,C,D
0,85.0,2,75.0,0
1,3.0,4,75.0,1
2,85.0,3,NaN,5
3,NaN,3,NaN,4


In [140]:
# iris data
# (1단계) 각 열마다 결측치를 20개씩
# 2-1단계 결측치는 각 열의 중위수로 대체 : fillna
# 2-2단계 결측치는 종별 열별 중위수로 대체: 
import seaborn as sns
import pandas as pd
import numpy as np
import random
iris = sns.load_dataset('iris')
iris.head(2)

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa


In [141]:
# 1단계
for col in range(iris.shape[1]-1):
    iris.iloc[random.sample(range(150),20),col] = np.nan

In [145]:
# 2-1단계
iris.fillna(iris.median(axis=0, numeric_only=True))

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,1.3,setosa
1,4.9,3.0,4.5,0.2,setosa
2,5.7,3.2,1.3,0.2,setosa
3,5.7,3.1,1.5,0.2,setosa
4,5.0,3.0,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.3,virginica
147,6.5,3.0,5.2,1.3,virginica
148,6.2,3.4,5.4,2.3,virginica


In [121]:
# 2-2단계
for row in range(iris.shape[0]):
    iris.iloc[row] =iris.iloc[row].fillna(iris.pivot_table(index='species', 
                                 values=iris.columns[:-1],
                                 aggfunc='median').loc[iris.iloc[row].species])

In [143]:
# 2-2단계
def iris_fillna(irisrow):
    irisrow = irisrow.copy()
    for col, value in irisrow.items():
        if isinstance(value, (int, float, np.number)) and np.isnan(value):
            irisrow[col] = iris_median.loc[irisrow.species, col]
    return irisrow

In [144]:
iris.apply(iris_fillna, axis=1)

KeyError: 'setosa'

## 12.2 replace(to_value, new_value,inplace=T/F)
- to_value를 new_value로 바꿈
- to_value에 정규표현식 이용
- 데이터프레임, 시리즈 적용

In [158]:
s = pd.Series([0,2,3,np.nan])
s.replace(np.nan,0)
s.replace([0,1],99) 
s.replace([0,1],[100,101])

0    100.0
1      2.0
2      3.0
3      NaN
dtype: float64

In [163]:
df = pd.DataFrame({'A':['bat','foo','bait'],
                   'B':['abc','bar','xyz']})
df.replace(r'ba.+','XX', regex=True)

,A,B
0,XX,abc
1,foo,XX
2,XX,xyz


## 12.3 mask(조건), where(조건) 
- where(조건) : 조건이  True인 요소만 출력, 그 외 요소는 결측치
- mask(조건)  : 조건이 False인 요소만 출력, 그 외 요소는 결측치

In [168]:
s = pd.Series(range(1,5))
s

0    1
1    2
2    3
3    4
dtype: int64

In [169]:
s.where(s>2)

0    NaN
1    NaN
2    3.0
3    4.0
dtype: float64

In [ ]:
s.mask(s>2)

## 12.4 dropna
- 결측치가 있는 데이터 누락(행, 열)

In [174]:
df = pd.DataFrame({'name':['홍','김','이',np.nan],
                   'age' :[np.nan, 20, 30, np.nan],
                   'born' :['서울','부산',np.nan,np.nan]})
df

,name,age,born
0,홍,NaN,서울
1,김,20.0,부산
2,이,30.0,NaN
3,NaN,NaN,NaN


In [175]:
df.dropna(how='all') # all 모두 결측치인 행만 삭제 , default : axis=0

,name,age,born
0,홍,NaN,서울
1,김,20.0,부산
2,이,30.0,NaN


In [176]:
df.dropna() #결측치가 하나라도 있는 행 삭제

,name,age,born
1,김,20.0,부산


In [177]:
df.dropna(axis=1) # 결측치가 하나라도 있는 열 삭제 

""
0
1
2
3


In [178]:
df.dropna(subset=['name','age'])

,name,age,born
1,김,20.0,부산
2,이,30.0,NaN


# 13절 시리즈
- 1차원 데이터

In [179]:
s = pd.Series([1000, 7000, 5000],
              index=['mango','apple','banana'])
s

mango     1000
apple     7000
banana    5000
dtype: int64

In [182]:
# 조회
s['mango'], s.mango

(1000, 1000)

In [185]:
# 수정
s['mango'] = 2800
# 추가
s['orange'] = 4000
s

mango     2800
apple     7000
banana    5000
orange    4000
dtype: int64

In [188]:
# 슬라이싱 가능 [from:to] from부터 to(포함)까지
s['mango':'banana'] 

mango     2800
apple     7000
banana    5000
dtype: int64

In [189]:
s[['mango','banana'] ]

mango     2800
banana    5000
dtype: int64

In [190]:
s.sort_index()

apple     7000
banana    5000
mango     2800
orange    4000
dtype: int64

In [192]:
s.sort_values() # defaul: 오름차순정렬

mango     2800
orange    4000
banana    5000
apple     7000
dtype: int64

In [193]:
s.sort_values(ascending=False) # 내림차순

apple     7000
banana    5000
orange    4000
mango     2800
dtype: int64

In [200]:
# 시리즈를 데이터프레임으로
s.to_frame()

,0
mango,2800
apple,7000
banana,5000
orange,4000


# 14절 연습문제

In [194]:
iris = sns.load_dataset('iris')

In [195]:
# 1
iris.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [222]:
# 2
X = iris.iloc[:,:-1]
y = iris.iloc[:,-1:]
X,y

(     sepal_length  sepal_width  petal_length  petal_width
 0             5.1          3.5           1.4          0.2
 1             4.9          3.0           1.4          0.2
 2             4.7          3.2           1.3          0.2
 3             4.6          3.1           1.5          0.2
 4             5.0          3.6           1.4          0.2
 ..            ...          ...           ...          ...
 145           6.7          3.0           5.2          2.3
 146           6.3          2.5           5.0          1.9
 147           6.5          3.0           5.2          2.0
 148           6.2          3.4           5.4          2.3
 149           5.9          3.0           5.1          1.8
 
 [150 rows x 4 columns],
        species
 0       setosa
 1       setosa
 2       setosa
 3       setosa
 4       setosa
 ..         ...
 145  virginica
 146  virginica
 147  virginica
 148  virginica
 149  virginica
 
 [150 rows x 1 columns])

In [207]:
# 3
temp = iris.head(50)
temp

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
5,5.4,3.9,1.7,0.4,setosa
6,4.6,3.4,1.4,0.3,setosa
7,5.0,3.4,1.5,0.2,setosa
8,4.4,2.9,1.4,0.2,setosa
9,4.9,3.1,1.5,0.1,setosa


In [340]:
# 4
temp.describe(include='float64')

,sepal_length,sepal_width,petal_length,petal_width
count,50.00000,50.000000,50.000000,50.000000
mean,5.00600,3.428000,1.462000,0.246000
std,0.35249,0.379064,0.173664,0.105386
min,4.30000,2.300000,1.000000,0.100000
25%,4.80000,3.200000,1.400000,0.200000
50%,5.00000,3.400000,1.500000,0.200000
75%,5.20000,3.675000,1.575000,0.300000
max,5.80000,4.400000,1.900000,0.600000


In [210]:
# 5
iris_versicolor = iris[iris.species=='versicolor'].reset_index()
iris_versicolor

,index,sepal_length,sepal_width,petal_length,petal_width,species
0,50,7.0,3.2,4.7,1.4,versicolor
1,51,6.4,3.2,4.5,1.5,versicolor
2,52,6.9,3.1,4.9,1.5,versicolor
3,53,5.5,2.3,4.0,1.3,versicolor
4,54,6.5,2.8,4.6,1.5,versicolor
5,55,5.7,2.8,4.5,1.3,versicolor
6,56,6.3,3.3,4.7,1.6,versicolor
7,57,4.9,2.4,3.3,1.0,versicolor
8,58,6.6,2.9,4.6,1.3,versicolor
9,59,5.2,2.7,3.9,1.4,versicolor


In [229]:
# 6
X.merge(y, left_index=True, right_index=True)

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


In [233]:
#7
iris.mean(numeric_only=True)

sepal_length    5.843333
sepal_width     3.057333
petal_length    3.758000
petal_width     1.199333
dtype: float64

In [235]:
# 8
iris.corr(numeric_only=True)

,sepal_length,sepal_width,petal_length,petal_width
sepal_length,1.000000,-0.117570,0.871754,0.817941
sepal_width,-0.117570,1.000000,-0.428440,-0.366126
petal_length,0.871754,-0.428440,1.000000,0.962865
petal_width,0.817941,-0.366126,0.962865,1.000000


In [265]:
# 9
avg = iris.mean(numeric_only=True)
iris.iloc[:,:-1] - avg

,sepal_length,sepal_width,petal_length,petal_width
0,-0.743333,0.442667,-2.358,-0.999333
1,-0.943333,-0.057333,-2.358,-0.999333
2,-1.143333,0.142667,-2.458,-0.999333
3,-1.243333,0.042667,-2.258,-0.999333
4,-0.843333,0.542667,-2.358,-0.999333
...,...,...,...,...
145,0.856667,-0.057333,1.442,1.100667
146,0.456667,-0.557333,1.242,0.700667
147,0.656667,-0.057333,1.442,0.800667
148,0.356667,0.342667,1.642,1.100667


In [269]:
#10
iris_mean = iris.pivot_table(index='species', 
                 values=iris.columns[:-1],
                 aggfunc='mean')
iris_mean

,petal_length,petal_width,sepal_length,sepal_width
species,,,,
setosa,1.462,0.246,5.006,3.428
versicolor,4.260,1.326,5.936,2.770
virginica,5.552,2.026,6.588,2.974


In [370]:
#11

iris_copy = iris.copy()
for row in range(iris.shape[0]):
    iris_copy.iloc[row,:-1] -= iris_mean.loc[iris.iloc[row].species]
iris_copy.groupby(by='species').head(3)  

iris.apply(lambda series : series - iris_mean.loc[series.species], axis=1)

,petal_length,petal_width,sepal_length,sepal_width,species
0,-0.062,-0.046,0.094,0.072,NaN
1,-0.062,-0.046,-0.106,-0.428,NaN
2,-0.162,-0.046,-0.306,-0.228,NaN
3,0.038,-0.046,-0.406,-0.328,NaN
4,-0.062,-0.046,-0.006,0.172,NaN
...,...,...,...,...,...
145,-0.352,0.274,0.112,0.026,NaN
146,-0.552,-0.126,-0.288,-0.474,NaN
147,-0.352,-0.026,-0.088,0.026,NaN
148,-0.152,0.274,-0.388,0.426,NaN
